In [1]:
import inv_dataset
import browser
import os
import pandas as pd

All Completed, End Task.


In [2]:
inventory_folder = 'data/inventory/'
udpate_folder_path = 'data/updates/'

#convert inventory xls to csv
inventory_files = inv_dataset.Converter()
inventory_files.to_csv(inventory_folder)

#convert updates xls to csv
update_files = inv_dataset.Converter()
update_files.to_csv(udpate_folder_path)

In [3]:
#read inventory csv into dataframe
inventory = inv_dataset.Reader()
inv_df = inventory.get_df(inventory_folder)

In [4]:
'''
back to parent directory if necessery
Use this line: 
os.chdir("../..")

check current directory:
os.getcwd()
'''

os.getcwd()

'c:\\Users\\Waddy Store\\Documents\\Code\\codespaces-flask'

In [5]:
def PreOrderClose():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            df = pd.read_csv(file_path, index_col=None, header=0)
            df = df.drop(index=0)

            df = df.loc[df['Preorder Product'] == "Y"]
            df = df.loc[df['Status'] == "Y"]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] > 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.close_preorder(process_list)   

In [6]:
PreOrderClose()

[['5f3b50bc09ec050048c12f99', '6971668340378', True], ['63734a8e89e4d9001cb8b2fc', '4526374213410', False]]
Clicker starts
Login sucessful
Now browsing to SKU: 5f3b50bc09ec050048c12f99
Now browsing to SKU: 63734a8e89e4d9001cb8b2fc
Go to Price and Quantity Tab
Checkbox Status is  None
pass clicking checkbox, already unchecked
Go to Settings Tab
No action, Switch alraedy off
Saved changes, completed


In [7]:
def PreOrderOpen():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            update_df = pd.read_csv(file_path, index_col=None, header=0)
            update_df = update_df.drop(index=0)

            '''
            match certain condition for open pre-order settings
            may add model or brand Keyword (case sensitive here)
            '''
            searchfor = [
                'SN-', 'ARCWAVE', 'HARU', 'LELO', 'Arcwave', 
                '小怪獸', 'BATTLEMAN', 'TS6', 'TITAN', 'ICHIKI','R20',
                'GINRO', 'Relove', 'We-Vibe', '草本精華', 'Womanizer Liberty',
                'Womanizer Starlet', 'Womanizer Premium 2', 'Satisfyer', 'Love Dear',
                'Womany', 'EXE Japanese Real Hole', 'Garden Costume', 'APS'
                ]

            df = update_df.loc[update_df['Product Name (Traditional Chinese)'].str.contains('|'.join(searchfor), na = False)]

            ''' end conidtion'''

            df = df.loc[df['Preorder Product'] == "N"]
            df = df.loc[df['Status'] == "Y"]

            discon_tag = 'dis'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag, na=False)]

            discon_tag_manual = 'shortage'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag_manual, na=False)]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] <= 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.open_preorder(process_list)   

In [8]:
PreOrderOpen()

[['5f3dd3871b3082002d76d59c', '4710006290096', False], ['5f86b5f17e5a17003b19a1dd', '4710006290171', False], ['5fa920e025751a00298c28fe', '48847824', False], ['5fa92235e075fb16c52c0271', '69356464', False], ['5fedd63400459d00381ef17e', '4049369016365', False], ['60488d2b7f0989001b684e3b', '4573423129635', False], ['606423686e5e80002f721490', '4573423129604', False], ['607c10fad96a2b002a4b83b7', '4573423129598', False], ['60a1f51c66cbe3003e3ccdf6', '4573423125927', False], ['60ae11dc73f8460017821da2', '4573423129611', False], ['60c8927e8a2a9b002908e5a8', '4580279018631', False], ['60c89772c765eb0014ef307a', '4573423119339', False], ['60e681301658610032056b54', '4582593580986', False], ['60f914af9d99680029d006f8', '4580279019829', False], ['6156cc7e3c09480014c7e9dc', '4573423119346', False], ['618a4d2ed89bf0003282bfa0', '4573423125958', False], ['618e9a67c517410017817214', '4710006290348', False], ['61c43df7e560bc0026fb716e', '4582593592293', False], ['62317658eda8c40014c5a641', '4582593